# 2 and 3ST using Mach relations

In [1]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
def phi(theta, M):
    out = optimize.fsolve(lambda x: fun(x,M,theta),0.01)
    out = np.append(out, optimize.fsolve(lambda x: fun(x,M,theta),np.pi/2))
    return out
def theta(x,M):
    out = 2*(1/np.tan(x))*(M**2*(np.sin(x))**2-1)/(M**2*(1.4+np.cos(2*x))+2)
    out = np.arctan(out)
    return out
def fun(x,M,theta):
    f=np.tan(theta)-2*(1/np.tan(x))*(M**2*(np.sin(x))**2-1)/(M**2*(1.4+np.cos(2*x))+2)
    return f
def fun2(x,M):
    theta=np.arctan(2*(1/np.tan(x))*(M**2*(np.sin(x))**2-1)/(M**2*(1.4+np.cos(2*x))+2))
    return theta
def thetaMax(M):
    phi=optimize.fmin(lambda x: -fun2(x,M),0.1)
    theta=fun2(phi,M)
    return theta

class state(object):
  def __init__(self,p=100000,M=1.5,T=300,R=287,phi=0,gamma=1.4):
    self.p=p
    self.M=M
    self.T=T
    self.gamma=gamma
    self.phi=phi
    self.R=R
    self.rho=self.p/(self.R*self.T)
    self.u=self.M*np.sqrt(self.gamma*self.R*self.T)
    
def oblique(state0,theta1=10*np.pi/180,weak='True'):
  #theta=theta
  p=phi(theta1,state0.M)
  tmax=thetaMax(state0.M)
  print("Theta max :",tmax*180/np.pi)
  if tmax<theta1:
        print("Shock detached")
  else:
    if weak=='True':
      ph=p[0]
    else:
      ph=p[1]
    M1=np.sqrt((1+(state0.gamma-1)*0.5*(state0.M*np.sin(ph))**2)/(state0.gamma*(state0.M*np.sin(ph))**2-(state0.gamma-1)*0.5))/(np.sin(ph-theta1))
    p2=state0.p*(1+2*state0.gamma*((state0.M*np.sin(ph))**2-1)/(1+state0.gamma))
    T2=state0.T*(p2/state0.p)*(2+(state0.gamma-1)*(state0.M*np.sin(ph))**2)/((1+state0.gamma)*(state0.M*np.sin(ph))**2)
    state1=state(M=M1,p=p2,T=T2)
    state1.phi = ph
    state1.theta= theta1
    return state1

def ST(state0,theta1):
  state1=oblique(state0,theta1=theta1)
  if theta1<thetaMax(state1.M):
        print("This case is a Regular reflection")
        print("Solving using 2 shock theory . . .")
        state2=oblique(state1,theta1=theta1)
        return state1,state2
  else:
    print("This case is a Mach reflection")
    print("Solving for the Standard solution using 3 shock theory...")
    phi = ThreeST(state0,state1,theta1)
    phi2 = phi[0]
    phi3 = phi[1]
    theta2 = theta(phi2,state1.M)
    print(theta2*180/np.pi)
    theta3 = theta(phi3,state0.M)
    state3 = oblique(state0,theta1=theta3)
    state3.phi = phi3
    state3.theta = theta3
    state2 = oblique(state1,theta1=theta2)
    state2.phi = phi2
    state2.theta =theta2
    return state1,state2,state3

def ThreeST(state0,state1,theta):
    p = np.array([[]])
    p = np.append(p,state0.M)  # M0
    p = np.append(p,state1.M)  # M1
    p = np.append(p,theta)  # theta
    p = np.append(p,state1.p/state0.p) # p1/p0
    return optimize.fsolve(lambda x: fun3(x,p),100*np.array([0.01,0.01]))
    
def fun3(x,p,gamma = 1.4):
    """
    x[0] - phi2
    x[1] - phi3
    """
    f=np.array([])
    f=np.append(f,(1+2*(p[1]**2*np.sin(x[0])**2-1)*gamma/(gamma+1))*p[3]-(1+2*(p[0]**2*np.sin(x[1])**2-1)*gamma/(gamma+1)))
    f=np.append(f,p[2]-theta(x[0],p[1])-theta(x[1],p[0]))
    return f
    
    
def polar(state,both = 'True', positive='True',gamma=1.4):
    tmax= thetaMax(state.M)
    theta = np.linspace(0,tmax,100)
    pWeak = np.asarray([pRatio(k,state.M) for k in theta])
    pStrong = np.asarray([pRatio(k,state.M,weak = 'False') for k in theta])
    theta = np.append(theta,np.flipud(theta))
    p = np.append(pWeak,np.flipud(pStrong))
    if both =='True':
       p = np.append(p,np.flipud(p))
       theta = np.append(theta,np.flipud(-theta))
    return theta, p

def plot(theta,p):
    for k,j in zip(theta,p):
        plt.plot(theta[k]*180/np.pi,p[j])
    plt.grid()
    plt.show()
  
   
    
def pRatio(x,M,gamma =1.4, weak = 'True'):
    y = phi(x,M)
    if weak =='True':
        z = y[0]
    else : 
        z = y[1]
    p = 1+2*gamma*((M*np.sin(z))**2-1)/(1+gamma)
    return p

def pRatioMax(M):
    return optimize.fmin(lambda x: -pRatio(x,M,gamma=1.4),0.1)

## 2 shock using mach relations

In [2]:
s0=state(M=2.5,p=100000, T = 300)
s1,s2=ST(s0,theta1=10*np.pi/180)

Optimization terminated successfully.
         Current function value: -0.520063
         Iterations: 20
         Function evaluations: 40
Theta max : [29.79744053]
Optimization terminated successfully.
         Current function value: -0.425732
         Iterations: 20
         Function evaluations: 40
This case is a Regular reflection
Solving using 2 shock theory . . .
Optimization terminated successfully.
         Current function value: -0.425732
         Iterations: 20
         Function evaluations: 40
Theta max : [24.39263962]


In [3]:
s1.M

2.0859286654960028

In [4]:
s1.p

186387.051818015

In [5]:
s1.T

360.92016750552096

In [6]:
s1.theta*180/np.pi

10.0

In [7]:
s1.phi*180/np.pi

31.850592231272163

In [8]:
s2.M

1.719673377848485

In [9]:
s2.p

322484.9554292899

In [10]:
s2.T

424.1400922499581

In [11]:
s2.theta*180/np.pi

10.0

In [12]:
s2.phi*180/np.pi

37.6825397452867

## 3 shock using mach relations

In [13]:
s0=state(M=3.5,p=100000, T = 300)
s1,s2,s3=ST(s0,theta1=36*np.pi/180)

Optimization terminated successfully.
         Current function value: -0.643451
         Iterations: 20
         Function evaluations: 40
Theta max : [36.86701013]
Optimization terminated successfully.
         Current function value: -0.050648
         Iterations: 21
         Function evaluations: 43
This case is a Mach reflection
Solving for the Standard solution using 3 shock theory...
-3.482010530981078e-11
Optimization terminated successfully.
         Current function value: -0.643451
         Iterations: 20
         Function evaluations: 40
Theta max : [36.86701013]
Optimization terminated successfully.
         Current function value: -0.050648
         Iterations: 21
         Function evaluations: 43
Theta max : [2.90192563]


In [14]:
s1.M

1.1593565635689003

In [15]:
s1.p

1057155.6313417251

In [16]:
s1.T

815.7175591025328

In [17]:
s1.theta*180/np.pi

36.0

In [18]:
s1.phi*180/np.pi

60.0902271616511

In [19]:
s2.M

1.1593565635704244

In [20]:
s2.p

1057155.631339664

In [21]:
s2.T

815.7175591020786

In [22]:
s2.theta*180/np.pi

-3.482010530981078e-11

In [23]:
s2.phi*180/np.pi

59.60382041083888

In [24]:
s3.M

1.1593565635928773

In [25]:
s3.p

1057155.6313275478

In [26]:
s3.T

815.7175590953844

In [27]:
s3.theta*180/np.pi

35.99999999981012

In [28]:
s3.phi*180/np.pi

60.090227161001216